In [54]:
import time
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib.request
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException

from datetime import date
import pandas as pd
date_today = date.today()

SCROLL_PAUSE_TIME = 3

#URL = 'https://www.pyermoss.com/collections/pyer-moss-collection-3-mens-all'
URL = 'https://www.pyermoss.com/collections/pyer-moss-collection-3-5-womens-all'
driver = webdriver.Chrome('/Users/mandili/.wdm/drivers/chromedriver/87.0.4280.20/mac64/chromedriver')
driver.get(URL)

product_url = []
name = []
picture_url = []
timestamp = []
brand = []
product_type = []
price = []
sales_price = []

last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

soup = BeautifulSoup(driver.page_source,'html.parser')

for i in soup.find_all('div',{'class':'bc-sf-filter-product-item bc-sf-filter-product-item-grid has-bc-swap-image bc-sf-filter-grid-width-4 bc-sf-filter-grid-width-mb-2'}):
    
    timestamp.append(date_today)
    brand.append('Pyer Moss')
    product_type.append("Women's Clothing")
    
    product_url.append("https://www.pyermoss.com" + i.a.get('href'))
    name.append(i.find('div',{'class':'bc-sf-filter-product-bottom'}).a.text)
    picture_url.append("https:" + i.find('div',{'class':'bc-sf-filter-product-item-image'}).img.get('data-src'))
    
    sales_price.append(i.find('p',{'class':'bc-sf-filter-product-item-price'}).text.strip())
    price.append(None)
    
for i in soup.find_all('div',{'class':'bc-sf-filter-product-item bc-sf-filter-product-item-grid has-bc-swap-image bc-sf-filter-grid-width-4 bc-sf-filter-grid-width-mb-2 sold-out'}):
        
    timestamp.append(date_today)
    brand.append('Pyer Moss')
    product_type.append("Women's Clothing")
    
    product_url.append("https://www.pyermoss.com" + i.a.get('href'))
    name.append(i.find('div',{'class':'bc-sf-filter-product-bottom'}).a.text)
    picture_url.append("https:" + i.find('div',{'class':'bc-sf-filter-product-item-image'}).img.get('data-src'))
    
    sales_price.append(i.find('p',{'class':'bc-sf-filter-product-item-price'}).text.replace('Sold Out',''))
    price.append(None)

# Second Layer

In [55]:
product_description = []
material = []
picture_name = []
color = []
counter = 1

for link in product_url:
    item_color = []
    item_material = []
    ourUrl = urllib.request.urlopen(link)
    soup = BeautifulSoup(ourUrl,'html.parser')
    product_description.append(soup.find('div',{'class':'rte'}).text.strip().replace('\n','').replace('\xa0',''))
    color.append(None)
    material.append(None)
    
    picture_name.append('pyer-moss-women_' + str(counter) + '.jpg') # change variable
    counter += 1

In [56]:
column_names = ["brand", "product_type", "product_name", 'product_description', "price", 
                "sales_price", "color","material", "product_url", "picture_url", "timestamp", "picture_name"]
second_pass = pd.DataFrame(columns = column_names)
second_pass.brand = brand
second_pass.product_type = product_type
second_pass.product_name = name
second_pass.product_description = product_description
second_pass.price = price
second_pass.sales_price = sales_price
second_pass.color = color
second_pass.material = material
second_pass.product_url = product_url
second_pass.picture_url = picture_url
second_pass.timestamp = timestamp
second_pass.picture_name = picture_name
second_pass.head()

,brand,product_type,product_name,product_description,price,sales_price,color,material,product_url,picture_url,timestamp,picture_name
0,Pyer Moss,Women's Clothing,Tall Mamba Boot,"﻿In collaboration with Brother Vellies, thista...",None,"$1,225",None,None,https://www.pyermoss.com/collections/pyer-moss...,https://cdn.shopify.com/s/files/1/2653/3618/pr...,2020-12-07,pyer-moss-women_1.jpg
1,Pyer Moss,Women's Clothing,Boat Neck Midi Silk Dress,This slim fitted silk dress features a high bo...,None,$975,None,None,https://www.pyermoss.com/collections/pyer-moss...,https://cdn.shopify.com/s/files/1/2653/3618/pr...,2020-12-07,pyer-moss-women_2.jpg
2,Pyer Moss,Women's Clothing,Tuxedo Shirt Dress,This elongated tuxedo style shirt dress featur...,None,$550,None,None,https://www.pyermoss.com/collections/pyer-moss...,https://cdn.shopify.com/s/files/1/2653/3618/pr...,2020-12-07,pyer-moss-women_3.jpg
3,Pyer Moss,Women's Clothing,Mamba Boot,"In collaboration with Brother Vellies, this an...",None,$825,None,None,https://www.pyermoss.com/collections/pyer-moss...,https://cdn.shopify.com/s/files/1/2653/3618/pr...,2020-12-07,pyer-moss-women_4.jpg
4,Pyer Moss,Women's Clothing,Oversized Scuba T-Shirt Dress,The mini scuba t-shirt dress offers an oversiz...,None,$350,None,None,https://www.pyermoss.com/collections/pyer-moss...,https://cdn.shopify.com/s/files/1/2653/3618/pr...,2020-12-07,pyer-moss-women_5.jpg


In [57]:
second_pass.to_csv("pyer-moss-women.csv", index=False, encoding = 'utf-8-sig')

# Download Images

In [58]:
import os
import requests as req

new_pd = pd.read_csv('/Users/mandili/Desktop/pyer-moss-women.csv', encoding='utf-8-sig')

os.mkdir('pyer-moss-women')
i = 1
for index, link in enumerate(new_pd['picture_url']):
    img_data = req.get(link).content
    with open('pyer-moss-women/'+ "pyer-moss-women_"+str(index+1)+'.jpg','wb+') as f:
        f.write(img_data)
    i+=1